# 002 线性回归的简洁实现

In [10]:
# import package
import numpy as np
import torch
from torch.utils import data
from torch import nn


## 构造一个人造的数据集

### 定义线性模型参数

- Weight: $w=[2, -3.4]^T$
- Bias: $b=4.2$
- 噪声: $\epsilon$
- 标签: $y = Xw + b + \epsilon$

In [11]:
def create_data(w, b, size):
    # X: 均值为0, 方差为1, 大小为size*len(w)
    X = torch.normal(0, 1, (size, len(w)))
    y = torch.matmul(X, w) + b
    # 创造噪音epsilon
    epsilon = torch.normal(0, 0.01, y.shape)
    y += epsilon
    # reshape(x, y): 将张量变形，值为-1时表示自动.
    return X, y.reshape((-1, 1))


real_w, real_b = torch.tensor([2, -3.4]), 4.2
features, labels = create_data(w=real_w, b=real_b, size=1000)

features[0], labels[0]


(tensor([ 1.3460, -0.2042]), tensor([7.5630]))

### 调用API读取参数

In [12]:
def load_data(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


batch_size = 10
data_iter = load_data((features, labels), batch_size)

next(iter(data_iter))


[tensor([[-0.5221, -0.5272],
         [-0.3410,  0.3007],
         [ 1.1999,  1.0283],
         [-1.3056,  1.4035],
         [-0.4998, -1.0790],
         [-0.9055,  0.8544],
         [-1.5474, -0.3380],
         [ 3.4811,  0.1132],
         [-0.8072, -0.4314],
         [ 0.0929, -1.6041]]),
 tensor([[ 4.9640],
         [ 2.4994],
         [ 3.1153],
         [-3.1794],
         [ 6.8618],
         [-0.5141],
         [ 2.2528],
         [10.7894],
         [ 4.0489],
         [ 9.8325]])]

## 定义模型

### 使用框架预定义好的层并初始化模型参数

In [13]:
net = nn.Sequential(nn.Linear(2, 1))
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)


tensor([0.])

### 定义Loss函数MSELoss

In [14]:
loss = nn.MSELoss()


### 实例化SGD实例

In [15]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)


## 开始训练

In [16]:
epoch_times = 10
for epoch in range(epoch_times):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch: {epoch+1}, loss: {l:f}')


epoch: 1, loss: 0.000309
epoch: 2, loss: 0.000100
epoch: 3, loss: 0.000100
